In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import asyncio
import logging
from datetime import datetime, timedelta
from src.pump_bot_tracker.strategy import wait_until


In [19]:
now = datetime.now()
target_time = now + timedelta(seconds=5)
await wait_until(target_time)

In [17]:
print(now)
print(target_time)

2025-05-08 12:39:38.099792
2025-05-08 12:40:38.099792


In [20]:
now

datetime.datetime(2025, 5, 8, 12, 42, 33, 786530)

In [21]:
target_time = now.replace(minute=29, second=0)

In [23]:
print(now)
print(target_time)

2025-05-08 12:42:33.786530
2025-05-08 12:29:00.786530


In [25]:
await wait_until(target_time)

In [2]:
.1087/.07606

1.4291348935051276

In [3]:
30101/0.025

1204040.0

In [5]:
30101/0.025

1204040.0

In [6]:
69676370/15.5

4495249.677419355

In [7]:
1558/1414

1.1018387553041018

In [2]:
import os
import argparse
import asyncio
from datetime import datetime
import logging
from dotenv import load_dotenv
from src.pump_bot_tracker.wallet_secrets import (
    WalletType,
    get_wallet_address,
    get_wallet_keypair,
)
from src.pump_bot_tracker.token_addresses import SOL
from src.pump_bot_tracker.blockchain import (
    convert_to_base_unit,
    transfer_sol,
    retry_swap_tokens,
    get_wallet_contents,
    wait_for_sol_diff,
)
from src.pump_bot_tracker.strategy import (
    ExitRules,
    poll_until_exit,
    start_keyboard_listener,
    watch_keyboard,
    wait_until
)

load_dotenv()
now = datetime.now()
# === CONFIG ===
TARGET_TIME = now.replace(
    hour=10, minute=57, second=0
)  # Set to initiate SWAP at 10:45:00
BASE_IDX = 0  # Index for the BASE wallet (trade this less frequently whenever I want to cash out or rebalance)
BOT_IDX = 0  # Index for this trade Eventually auto increment once I have a listener
MAX_SOL = 0.2  # Maximum SOL to devote to strategy

SLIPPAGE_BPS = 4500  # 45% slippage
PRIORITIZATION_FEE = 100000
RENT_BUFFER = (
    5000000  # Rent factor (in lamports) to be left behind on all SOL transactions
)

exit_rules = ExitRules(
    max_duration_s=60 * 9,  # 20 min (from the TARGET TIME)
    take_half_at=60,  # 60% gain
    take_all_at=250,  # 250% gain
    stop_out_at=-30,  # get completely out at -30%
    polling_interval=5,  # 5s intervals
)

KEYBOARD_EXIT_MAPPING = {
    "h": "sell_half",
    "a": "sell_all",
}

# Get wallets for this strategy:
base_phrase = os.getenv("SOLANA_MEME_PUMP_BASE")
trade_phrase = os.getenv("SOLANA_MEME_PUMP_TRADE")
base_bot_address = get_wallet_address(
    wallet_type=WalletType.BOT, mnemonic_phrase=base_phrase, wallet_idx=BASE_IDX
)
base_bot_keypair = get_wallet_keypair(
    wallet_type=WalletType.BOT, mnemonic_phrase=base_phrase, wallet_idx=BASE_IDX
)
trade_bot_address = get_wallet_address(
    wallet_type=WalletType.BOT, mnemonic_phrase=trade_phrase, wallet_idx=BOT_IDX
)
trade_bot_keypair = get_wallet_keypair(
    wallet_type=WalletType.BOT, mnemonic_phrase=trade_phrase, wallet_idx=BOT_IDX
)


In [3]:
from src.pump_bot_tracker.wallet_secrets import get_wallet_keypair, WalletType
TEST_WALLET = os.getenv("SOLANA_PUBLIC_KEY")
TEST_WALLET


'FXLLEYZBnN1y7FvHYWQzkviY45whLKPrZhcqkE1H91Gj'

In [4]:
wallet_contents = await get_wallet_contents(TEST_WALLET)
wallet_sol = wallet_contents.get(SOL, {}).get("raw_amount", 0)
test_keypair = get_wallet_keypair(WalletType.TEST)
print(wallet_sol)
wallet_contents

159642588


{'So11111111111111111111111111111111111111112': {'mint': 'So11111111111111111111111111111111111111112',
  'raw_amount': 159642588,
  'amount': 0.159642588,
  'decimals': 9,
  'name': 'Solana',
  'symbol': 'SOL'},
 'BxMZLMzVKsGJAdEN3sqsHmHqFeTbhLYZamiVuEQCpump': {'mint': 'BxMZLMzVKsGJAdEN3sqsHmHqFeTbhLYZamiVuEQCpump',
  'raw_amount': 27123036813,
  'amount': 27123.036813,
  'decimals': 6,
  'name': 'Kanye West Token',
  'symbol': 'KYT'}}

In [10]:
0.00001591/0.000003736

4.258565310492505

In [9]:
signature = await transfer_sol(
    test_keypair, base_bot_address, 10_000_000
    )
signature

'5CvpqcLE4cZuUbBtqxQejn5izHi1kJr77g4gTr8LY1ppYZoEoQR833rMe9qay15xyjumahUCYc5NWLdMLXWaamGR'

In [13]:
signature

'NtHCLs6P92gWPop2VAfmiVEJ79B34DfgurczuYAH7MjSGjNtX6Z3B4hSD1gxEv7dcdUkcj3VKtQa23jcMg5xqaK'

### Fast liquidity checking:

In [ ]:
import asyncio
import json
import websockets

RAYDIUM_AMM_V4 = "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8"
SEARCH_STRING = "initialize2"

async def main():
    uri = "wss://api.mainnet-beta.solana.com/"  # replace with your preferred RPC if needed

    async with websockets.connect(uri) as ws:
        sub_req = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "logsSubscribe",
            "params": [
                {
                    "mentions": [RAYDIUM_AMM_V4]
                },
                {
                    "commitment": "finalized"
                }
            ]
        }
        await ws.send(json.dumps(sub_req))
        print("Subscribed to Raydium logs...")

        while True:
            response = await ws.recv()
            data = json.loads(response)

            if "result" in data:
                continue  # Initial subscription confirmation

            try:
                logs = data["params"]["result"]["value"]["logs"]
                sig = data["params"]["result"]["value"]["signature"]
                if SEARCH_STRING in " ".join(logs):
                    print("New pool tx:", sig)
            except KeyError:
                pass

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [27]:
trade_bot_address

'8TRipPeB91R8uuajGBzEgcoBZu2dp53JCDyayjQAcsd4'

### Go from signature to transaction details:

In [39]:
from src.pump_bot_tracker.blockchain import get_helius_url
import requests
signature = "3nWjLLkzeEv4Ehs2nkLVxtukAjGqyynVeSRNrX1HTddrDopgLoCPmFF7BZAEHuTyVQ6Xcxr8yYLBsAjmv69uZWJa"
def get_transaction(signature: str):
    rpc_url = get_helius_url()
    payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTransaction",
            "params": [
                signature,
                {
                    "encoding": "jsonParsed",
                    "commitment": "confirmed",
                    "maxSupportedTransactionVersion": 0
                }
            ]
        }
    response = requests.post(rpc_url, json=payload)
    response.raise_for_status()
    return response.json()

In [40]:
data = get_transaction(signature)

In [41]:
data

{'jsonrpc': '2.0',
 'result': {'blockTime': 1746721645,
  'meta': {'computeUnitsConsumed': 57250,
   'err': None,
   'fee': 244000,
   'innerInstructions': [{'index': 4,
     'instructions': [{'parsed': {'info': {'extensionTypes': ['immutableOwner'],
         'mint': 'GtWDtbTw8oJqR6VDYPmDVvCoD3nfPQCoooABYQzAHNUG'},
        'type': 'getAccountDataSize'},
       'program': 'spl-token',
       'programId': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
       'stackHeight': 2},
      {'parsed': {'info': {'lamports': 2039280,
         'newAccount': '43v51JYdbu5cVdQHQ35CPa1qkxGENrNan6qgNfQBzR6k',
         'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
         'source': '2B1qGh1AwG9zv5ZWoww7kkCWRF5ePnC3cRgmyhja2GEB',
         'space': 165},
        'type': 'createAccount'},
       'program': 'system',
       'programId': '11111111111111111111111111111111',
       'stackHeight': 2},
      {'parsed': {'info': {'account': '43v51JYdbu5cVdQHQ35CPa1qkxGENrNan6qgNfQBzR6k'},
        'type'

In [42]:
data.keys()

dict_keys(['jsonrpc', 'result', 'id'])

In [52]:
from src.pump_bot_tracker.liquidity_pools import find_raydium_pool_by_mints, get_vault_balance, get_vault_addresses

In [44]:
mintA = "GtWDtbTw8oJqR6VDYPmDVvCoD3nfPQCoooABYQzAHNUG"
mintB = "So11111111111111111111111111111111111111112"

In [45]:
pool = find_raydium_pool_by_mints(mintA, mintB)

In [46]:
pool

{'amm_id': '3a5VXkeT3WZXgX5r28KJEZbFcbaPQFQ62CiWNtYWK13N',
 'lp_mint': 'Bu5oNnXBsjpcazBUwM32pQTMkf9ak3d2BT7FXrebDmSu',
 'base_mint': 'So11111111111111111111111111111111111111112',
 'quote_mint': 'GtWDtbTw8oJqR6VDYPmDVvCoD3nfPQCoooABYQzAHNUG',
 'market_id': '69BhUTCqPZPtSeCoHbbkLw7vE64e8NqRUNHFZRHViT1n',
 'version': 5}

In [54]:
vault_address = pool['amm_id']
vault_data = get_vault_addresses(vault_address)
vault_data

{'base_mint': 'So11111111111111111111111111111111111111112',
 'quote_mint': 'GtWDtbTw8oJqR6VDYPmDVvCoD3nfPQCoooABYQzAHNUG',
 'base_vault': 'CaUPaycN7YGmnbVSvaZKfheuzXJAKJ2qJ76fE2hQA6Gx',
 'quote_vault': '4m3oVvUP8ZwfjufdDpgk4LjFkYqZPsoV15V8Zw3eWXp7'}

In [56]:
sol_balance = get_vault_balance(vault_data['base_vault'])
meme_balance = get_vault_balance(vault_data['quote_vault'])


In [62]:
total_sol = sol_balance/10**9


In [63]:
total_sol * 183 * 2

1585.1604818879998

In [64]:
from src.pump_bot_tracker.websocket import listen_for_transaction

In [65]:
wallet = os.getenv("SOLANA_PUBLIC_KEY")


In [67]:
wallet
search_string = 'transfer'

In [ ]:
signature = await listen_for_transaction(wallet, search_string)

Subscribed to logs for wallet: FXLLEYZBnN1y7FvHYWQzkviY45whLKPrZhcqkE1H91Gj


In [69]:
signature

<coroutine object listen_for_transaction at 0x11a6c83d0>

In [11]:
from src.pump_bot_tracker.wallet_secrets import generate_wallets_from_mnemonic
from dotenv import load_dotenv
import os
mnemonic_phrase = os.getenv('SOLANA_MEME_PUMP_BASE')


In [12]:
from src.pump_bot_tracker.wallet_secrets import get_wallet_address, WalletType
get_wallet_address(WalletType.BOT, mnemonic_phrase, wallet_idx=0)


'3cSvQygypScaJb2bJHfdzXwwWmayvNin37Go8AkSTysS'

### Go from a signature to transaction information:

THESE are from a scammer wallet. Use as examples
Signature: 5qNc3FNfiMavgiDRxFJUjFozPJocehTjSZdpDWABaHPAd4vcn7kXmTx1dALBhsMonuW2o9Awg2csexiasCSiRcif
Signature: 2UNcW39BPCH5hrRUVo7Vz4GFEaD8RSFMyoAwyJP2kyAc1DxEUhZhWhqhDQp7eAsc8Vmi8vD3bEgP1T9FksN56CUz
Signature: d2qLPyaajCBey2tCZhsme6GNmA7zGRH8LMEhbr7fW7r6K2Uuv3nv4MPm6nnrm6DMmTui6SATctgygEWiJhKHudu
Signature: 4r7jD1HVwthKxg8yeQ5wovpLYpjWg3kJhMgDW841GMcXsMcXsBMgpBTJDu9a8RwatB9qHZgLFK1oT9KnPkwCk8KE
Signature: 5bdoMJ9UFoJJgPDZzPH25Kp9vajxxAxYRR6gB5FRqTEDfVKTRcmvDaU1FXohn29oXziYzq8CqqWyUsuffPoWWQKV
Signature: 28UrsC3ESVBTz9rfMEsYvTvvv9moodtjr5SxRhLUQ89AoWmm74RDNiPo4df1uit2xpPzfjYFGmEcrvbKq5w25A2Z
Signature: jbc47GFCjbqt4yer3R8BFCyQwju3S2YeBboenVYz4Ah9WNo8GyryzL26F9ebtnW97t13MBFBkGvQjZdpfEarQ9g
Signature: 2UxgZAoouhH167V8bNrrK4KSB2q1LE3qtQGHRu28FdD5EaiQysq6TA9Q8UpPgdRQcpvv835JpXiKt8bGMCP3MB9N
Signature: 5JmdCCrvBp6kQgfqA45F6yWQmupSMUaFbGf8N8sK7buQw3K5TqiGWrSG3RuRL8CdNygvd4J3mG1MgWbvRiXYEYp9
Signature: 3i5X7EctHEDyoLTLd5h9sjGPoR3emvpUonC7GGoPXJhaKqZVfs9PEZAq6S5YHJU5LKNUxKAP7XNwPb6MbE9tbi39
Signature: 4Uq9GJnvs7EazrU7m2WZEZEKoGJdUtmnEDq4EVW7N9XdymU8A915oeuybQhtocPNsqLTZcyL7j6CYHjThu4pFnSE
Signature: 21YamDaYcTuiKjJXCy6QJR9sZPJFTRvPcSwZ4iWsquq9FwCD5w7fYBLAgR7KBRG1sMeCz7XQ7CndLKTvZxG5pgHy
Signature: 5Usz397Zy5pMGViaHeAaSwFAd3S5BoKfNjPJP8T6rKAMm7985aQfyPN4cYFXFMT8PweavqsA5q5SGUbCKztoZ5sY
Signature: 3Vdezw5w5Mg1FWN889QLhJ3k7NNDxJjYP4Ve7yxzm4DvFuRa8qFoJCdBVgCqDVdwnKuc5c8fbSBpovve16vpMPj2

In [ ]:
from src.pump_bot_tracker.blockchain import get_helius_url
import requests
def get_transaction(signature: str):
    helius_url = get_helius_url()
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTransaction",
        "params": [
            signature,
            {
                "encoding": "jsonParsed",  # Use "json" if you prefer raw binary
                "maxSupportedTransactionVersion": 0
            }
        ]
    }
    response = requests.post(helius_url, json=payload)
    response.raise_for_status()
    return response.json()["result"]


In [57]:
# transfer_signature = "2D2UX25ZTKvcyqAFitBR1qttahyqfk6Skr8qBFNCc3R7jCGDkGQVvvHpr8P6oXTDxKvZEpct1bd6jxCXc5cNoxe7" # this is a transfer to a trading wallet
# transfer_data = get_transaction(transfer_signature)
# swap_signature = "56VJonrVVJmkwzEp5axNJuuxrwkXEmjZa1CoD1WbW3oPe6FHuG4NXcSrA25mHUg4pZDDQ785j8wkggW6LKMBxbed" # target account swapped SOL for TOKEN
# swap_data = get_transaction(swap_signature)
# I want the from and to wallet id as well as the tokens and 
liquidity_signature = "No7wTMsNoCWGa569pBWCUpHU6pHcGgURdT7xSV3brtZq4KZMEdNvazvjWJ7kNxQgbEGAPWJCkGMcHTts3tciASe" # Added liquidity to a token pair
liquidity_data = get_transaction(liquidity_signature)

<Response [200]>


In [60]:
def classify_transaction_type(tx: dict) -> str:
    """
    Classifies a transaction as 'transfer', 'swap', or 'mint'.
    """
    if tx["meta"].get("err") is not None:
        return "failed"

    message = tx["transaction"]["message"]
    instructions = message.get("instructions", [])
    inner_instructions = tx["meta"].get("innerInstructions", [])

    # Check for known swap programs # PUT CONSTANTS SOMEWHERE
    swap_programs = {
        "Jupiter": {"JUP4Fb2cqiRUcaTHdrPC8h2gNsA2ETXiPDD33WcGuJB"},
        "Raydium": {"RVKd61ztZW9GdKzYjYqYcsGZ3jQMMQ3jYbjuTyDqfFM", "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8"},
        "Orca": {"9WwG7FvC9mWM2yFvYJYp9hRtaRGNVNfFSczrjCjAYaWx"},
        "Meteora": {"meteobHSyKeEBgVJE6A36qvbmJzJxTHBvhPH7U8L7zj"}
    }

    all_program_ids = {ix.get("programId") for ix in instructions}
    for inner in inner_instructions:
        all_program_ids.update(ix.get("programId") for ix in inner.get("instructions", []))

    for protocol, program_ids in swap_programs.items():
        if all_program_ids & program_ids:
            return "swap"

    # Otherwise, fallback to checking system transfer
    for ix in instructions:
        if ix.get("programId") == "11111111111111111111111111111111":
            parsed = ix.get("parsed", {})
            if parsed.get("type") == "transfer":
                return "transfer"

    return "unknown"

def get_transaction_type_keywords(tx: dict) -> list[str]:
    """
    Fetches the transaction and extracts the instruction 'type' fields from parsed instructions.
    Returns a list of unique type keywords (e.g., 'transfer', 'mintTo', etc.)
    """


    type_keywords = set()

    # Check main instructions
    instructions = tx["transaction"]["message"].get("instructions", [])
    for ix in instructions:
        parsed = ix.get("parsed", {})
        type_ = parsed.get("type")
        if type_:
            type_keywords.add(type_)

    # Check inner instructions too
    inner = tx["meta"].get("innerInstructions", [])
    for group in inner:
        for ix in group.get("instructions", []):
            parsed = ix.get("parsed", {})
            type_ = parsed.get("type")
            if type_:
                type_keywords.add(type_)

    return list(type_keywords)

In [62]:
get_transaction_type_keywords(swap_data)

['createAccountWithSeed',
 'transfer',
 'initializeAccount3',
 'initializeImmutableOwner',
 'createIdempotent',
 'getAccountDataSize',
 'initializeAccount',
 'createAccount',
 'closeAccount']

In [64]:
get_transaction_type_keywords(transfer_data)

['transfer']

In [65]:
get_transaction_type_keywords(liquidity_data)

['createAccountWithSeed',
 'transfer',
 'create',
 'initializeAccount3',
 'transferChecked',
 'mintTo',
 'syncNative',
 'initializeImmutableOwner',
 'getAccountDataSize',
 'initializeAccount',
 'createAccount',
 'initializeMint2',
 'closeAccount']

In [58]:
classify_transaction_type(liquidity_data)

'transfer'

In [59]:
liquidity_data

{'blockTime': 1746979436,
 'meta': {'computeUnitsConsumed': 161270,
  'err': None,
  'fee': 80001,
  'innerInstructions': [{'index': 4,
    'instructions': [{'parsed': {'info': {'lamports': 1461600,
        'newAccount': '8ViS14h7ZWm1AtMVxRQLvAN38Av14PwzjZFWNcfkTC2i',
        'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
        'source': 'EDwBVBqYXky9oX7MBPcNz4itJmaShAprAZy9ufCvNH1K',
        'space': 82},
       'type': 'createAccount'},
      'program': 'system',
      'programId': '11111111111111111111111111111111',
      'stackHeight': 2},
     {'parsed': {'info': {'decimals': 9,
        'mint': '8ViS14h7ZWm1AtMVxRQLvAN38Av14PwzjZFWNcfkTC2i',
        'mintAuthority': 'GpMZbSM2GgvTKHJirzeGfMFoaZ8UR2X7F4v8vHTvxFbL'},
       'type': 'initializeMint2'},
      'program': 'spl-token',
      'programId': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
      'stackHeight': 2},
     {'parsed': {'info': {'account': '47tYTxseapfNbiszhXzszgTZr7KK6GrjQSqwDiPciY3B',
        'mint': '8V

In [46]:
swap_data

{'blockTime': 1746980971,
 'meta': {'computeUnitsConsumed': 60250,
  'err': None,
  'fee': 46000,
  'innerInstructions': [{'index': 4,
    'instructions': [{'parsed': {'info': {'extensionTypes': ['immutableOwner'],
        'mint': '6Bu6VAt1sQwaURGvE3p4kcLKXzfyAbMjhBQcDgkRZyvi'},
       'type': 'getAccountDataSize'},
      'program': 'spl-token',
      'programId': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
      'stackHeight': 2},
     {'parsed': {'info': {'lamports': 2039280,
        'newAccount': 'G841JQVJ765nxfNoYKzp9gQjjYLQWQuqWPE7SomripN7',
        'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
        'source': '5KQd1VGnHBoAaxZWzckRbVZkGJPgFYoasNFjuDk4qBiZ',
        'space': 165},
       'type': 'createAccount'},
      'program': 'system',
      'programId': '11111111111111111111111111111111',
      'stackHeight': 2},
     {'parsed': {'info': {'account': 'G841JQVJ765nxfNoYKzp9gQjjYLQWQuqWPE7SomripN7'},
       'type': 'initializeImmutableOwner'},
      'program': 'spl

In [67]:
2932/1037

2.8273866923818707

In [69]:
base_phrase = os.getenv("SOLANA_MEME_PUMP_BASE")
trade_phrase = os.getenv("SOLANA_MEME_PUMP_TRADE")
BASE_IDX = 0
base_bot_address = get_wallet_address(
    wallet_type=WalletType.BOT, mnemonic_phrase=base_phrase, wallet_idx=BASE_IDX
)

In [70]:
base_bot_address

'3cSvQygypScaJb2bJHfdzXwwWmayvNin37Go8AkSTysS'

In [71]:
1055/788.6

1.3378138473243724